## Inits

+ Standard modules:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from time import time

+ My own modules:

In [3]:
import input_load
from input_load import *

import split
from split import DataSplitter

import model_train
from model_train import ModelTrainer

import trigger
from trigger import *

import date_util
from date_util import *

+ Helpers:

In [5]:
def rand_int_in(low, high): # include both low & high
    return np.random.randint(low, high + 1, 1)[0]

## Load global vars

In [4]:
CLEAN_DIR = '~/data/venture=Zalora/sg/clean/'
RES_DIR = '~/projects/daas-markdown/res/'
group_dir = CLEAN_DIR + 'groups/'
feat_dir = group_dir + 'feats/'
# TODAY = pd.datetime.today().date()

# Preparations

This stage includes following tasks:

+ find configs to be optimized
+ calculate possible prices for the configs
+ train a sales predictor which will predict sales associated with the prices

## Configs to be optimized

Given a group of SKU configs, we want to detect which SKU configs we need to optimize prices. Those are configs which are either selling __too slow__ or __too fast__. Thus, we can detect those by comparing target sale velocity with recent sale velocity (in two recent weeks). This is implemented in our `trigger` module.

## Possible prices 

We now calculate possible prices for each config $cf$. Those are prices discounted from the black price of $cf$, starting from black price minus $5\%$, each time we discount $5\%$. Thus, the possible prices will have the following form:

$$
black\_price(cf) - k * unit\_discount, k \in 1,\dots, 16
$$
where
$$
unit\_discount = 0.05 * black\_price(cf)
$$

In [ ]:
# moved to cls PriceAccessor
def get_correct_price(cf, tmp_rrp):
    #     in case of several black prices, use the latest one (from mtg on 10 Nov 2017)
    sub_df = tmp_rrp[tmp_rrp.sku_config == cf]
    sub_df = sub_df.drop_duplicates('black_price').sort_values('snapshot_date', ascending=False)
    
    if sub_df.shape[0] > 1: # for debug
        print(sub_df)
    
    latest_price = sub_df['black_price'].iat[0]
    return latest_price

def query_black_price(configs, group_df):
    
    print('Querying black prices...')
    print('Below are SKUs with changing black price, we are using the latest one for black price')
    
    cols = ['sku_config', 'snapshot_date', 'price']
    tmp_rrp = group_df[group_df.sku_config.isin(configs)][cols].drop_duplicates()
    tmp_rrp = tmp_rrp.rename(columns={'price': 'black_price'})
    
    correct_prices = [get_correct_price(cf, tmp_rrp) for cf in configs]
    return pd.DataFrame({'sku_config': configs, 'black_price': correct_prices})

__Notes:__ 

+ There are SKUs with too large black prices. These must be errors from data input.
+ some SKUs have more than $1$ black price. How to handle these cases? Should we just take the smallest price?

Use latest black price (mtg on 10 Nov 2017)

__Note__: for prototype purpose, I try only 3 possible values for each config. Later will try a full range.

In [ ]:
# moved to cls PriceAccessor
ks = np.arange(0, 3) + 1
def cal_possible_prices(cf, rrp_df, count=1, n_digit=2): # round to 2 decimals
    if (count % 100) == 0:
        print('{} configs and counting...'.format(count))
    
    black_price = rrp_df.query('sku_config == "{}"'.format(cf))['black_price'].iat[0]
    unit_discount = 0.05*black_price
    return np.round(black_price - ks*unit_discount, decimals=n_digit)

## Train a sales predictor

In [219]:
reload(demand_pred)
from demand_pred import *

In [220]:
demand_predictor = DemandPredictor()

In [32]:
feat_mat = in_loader.form_feat_mat(group=gname, global_X=global_X)
ready_df = in_loader.attach_response(feat_mat, my_trigger.sale_hist)
train_df, test_df = data_splitter.split_train_test(ready_df)

Loading features of group dresses_female_autumn-winter from /Users/gfg/data/venture=zalora/sg/clean/groups/feats/dresses_female_autumn-winter.csv...
	 dropped NA configs
Obtained feature matrix of group dresses_female_autumn-winter
	 info on the matrix
<class 'pandas.core.frame.DataFrame'>
Int64Index: 160080 entries, 0 to 160079
Data columns (total 17 columns):
sku_config           160080 non-null object
current_price        160080 non-null float64
discount_from_rrp    160080 non-null float64
n_competitor         160080 non-null int64
rel_price            160080 non-null float64
product_name         160080 non-null object
color                160080 non-null object
color_pop            160080 non-null float64
sub_cat              160080 non-null object
brand_name           160080 non-null object
brand_type           160080 non-null object
date                 160080 non-null object
total_views          160080 non-null int64
total_impressions    160080 non-null int64
mean_views         

In [33]:
import gc

In [34]:
# del global feat matrix to free mem for training
del global_X
gc.collect()

433

In [221]:
best_xgb = demand_predictor.fit_xgb(train_df)

Fitting XGBoost to train data...
()
Performing CV for xgboost...
Fitting 3 folds for each of 40 candidates, totalling 120 fits
-0.2060728528
{'n_estimators': 20, 'max_depth': 3}


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   40.6s finished


In [341]:
predictor = best_xgb.best_estimator_

## Predict sales associated with a value of total price

Given the trained estimator (predictor), we now can perform the sales prediction needed by PO.

__Context__: Consider date $dd$. Let `group_size` be the number of configs in the group on $dd$ and $s_{out}$ be the total price of configs that we do not need to be optimized (thus the total price is fixed).

Given a config, possible prices and the fact that the prices sum to $s$, predict sales of the config at the given prices.

In [335]:
# context/global vars for the sale predictor
group_size =  len(all_configs)
s_out = 1000 # hypothetic, replace by actual value later
mean_price = (s + s_out)/group_size
cols = ['sku_config', 'price', 'total_price', 'predicted_sale']

In [350]:
def predict_sales(cf, prices, s):
    print('config: {}'.format(cf))
    data_cols = feats + ['black_price']
    
    def predict_sale_for_price(p):
        rec = feat_mat.query('sku_config == "{}"'.format(cf))[data_cols].head(1)
        rec['date'] = dd

        # update price-relevant feats
        rec['current_price'] = p
        rec['discount_from_rrp'] = 100*(rec['black_price'] - rec['current_price'])/rec['black_price']
        rec['rel_price'] = rec['current_price']/mean_price
        return predictor.predict(rec[feats])[0]
    
    sales = [predict_sale_for_price(p) for p in prices]
    res = pd.DataFrame({'sku_config': cf, 'price': prices, 'total_price': s, 'predicted_sale': sales})
    return res[cols]

In [338]:
# Test
frames = [predict_sales(cf, test_prices, s) for cf in test_configs]
pred_sale_df = pd.concat(frames)

config: 9EB52AA5DA4D3DGS
config: D4094AA895E68CGS
config: 1F22DAA71C149FGS


# Find global optimum prices

For each value of $s$, we solve the corresponding ($IP_s$) to find the local optimum prices $P^*_s$. Then we determine the global optimum prices $P^*$ as the one that provides maximum revenue among these $P^*_s$.

## Solve $IP_s$ for local optimum

For each possible value of $s$, I follow the steps in algorithm 1 in the [work](http://www.hbs.edu/faculty/Publication%20Files/kris%20Analytics%20for%20an%20Online%20Retailer_6ef5f3e6-48e7-4923-a2d4-607d3a3d943c.pdf):

+ solve relaxed $LP_s$ of $IP_s$ (e.g. integer constraints are removed)
+ calculate the lower bound for objective value of $IP_s$ via objective value of $LP_s$

In [388]:
# only needed when there are changes in code base
reload(input_load)
from input_load import *

In [389]:
in_loader = InputLoader(DataPrep())

Loading global feats in yr 2017 from file /Users/gfg/data/venture=zalora/sg/clean/2017/glob_feat.csv...


In [364]:
import lp_builder
from lp_builder import LP_builder

In [376]:
reload(lp_builder)
from lp_builder import LP_builder

### Build relaxed $LP_s$ for each value $s$ of total price

In [ ]:
gname = 'tops_female_autumn-winter'

lpb = LP_builder(in_loader, group=gname, feats=feats, sale_predictor=predictor)
prob = lpb.populateLP(s, dd, test_configs, test_prices, inv_amount, binary=False)

### Solve relaxed $LP_s$

In [362]:
print()
print('Started solver')
prob.solve()

Populating $LP_94.23$...
+ set variable types as ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
+ added bounds
+ added 3 one-price constraints for 3 configs
+ added sum constraint
# configs in group on date 2017-03-12: 891
# slow configs to be optimized: 3
Mean price of whole group: 55.25
Predicting sales for the slow configs given possible prices...
config: 9EB52AA5DA4D3DGS
config: D4094AA895E68CGS
config: 1F22DAA71C149FGS
+ added objective function
Type of problem: LP

Started solver
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =           125.122692


In [411]:
prob.solution.get_status()

1

In [412]:
prob.solution.get_values()

[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.49999999999999944, 0.0, 0.5000000000000006]

In [ ]:
prob.solution.get_objective_value()

In [413]:
prob.variables.get_names()

['x_11', 'x_12', 'x_13', 'x_21', 'x_22', 'x_23', 'x_31', 'x_32', 'x_33']

### Calculate lower bound for $IP_s$

## Find global optimum

# Tests

In [22]:
reload(trigger)
from trigger import *

reload(date_util)
from date_util import *

reload(split)
from split import DataSplitter

In [ ]:
my_trigger = Trigger(in_loader)
data_splitter = DataSplitter(split_ratio=0.8)

## Find configs to be optimized
Via `trigger`

In [421]:
gname = 'dresses_female_autumn-winter'
fname = feat_dir + gname + '.csv'

In [422]:
feat_mat = pd.read_csv(fname)
all_configs = feat_mat.sku_config.unique()

In [25]:
# load group sales history
group_sales = my_trigger.query_group_sales(gid)

fd, ld = date_range(group_sales)
dd = ld - timedelta(days=30)

Querying sale history of group dresses...


In [ ]:
#NOT run, this has some bug
slow_sku_df = my_trigger.find_slow_configs(gid, group_sales, dd)
print(slow_sku_df.head())

In [56]:
# slow_configs = slow_sku_df['sku_config'].unique()
slow_sku_df = pd.read_csv(RES_DIR + 'slow_sku.csv')
slow_configs = sorted(slow_sku_df['sku_config'].unique())
print('# slow configs: {}'.format(len(slow_configs)))

# slow configs: 254


In [57]:
configs = np.random.choice(slow_configs, size=10)

### Pick just a few SKUs for testing

In [ ]:
test_configs = list(slow_df.query('black_price == 34.9')['sku_config'].head(3))
test_prices = [33.16, 31.41, 29.66]

In [420]:
test_configs

['9EB52AA5DA4D3DGS', 'D4094AA895E68CGS', '1F22DAA71C149FGS']

In [ ]:
s = sum(test_prices)

In [116]:
# choose a date for testing, need to have all test configs
tmp = feat_mat[feat_mat.sku_config.isin(test_configs)].sort_values('date', ascending=False)
tmp.date.value_counts()

dd = pd.to_datetime('2017-03-12').date()

## Set possible prices

In [67]:
# moved to cls PriceAccessor
def load_group_snapshots(gname, group_dir):
    date_cols = ['snapshot_date', 
                 'special_from_date', 'special_to_date', 
                 'permanent_markdown_from_date']

    return pd.read_csv(group_dir + gname + '.csv', parse_dates=date_cols)

In [112]:
# moved to cls PriceAccessor 
def set_possible_prices(configs, group_df):
    # get correct black price
    rrp_df = query_black_price(configs, group_df)

    # set possible prices
    possible_prices = [ cal_possible_prices(cf, rrp_df, count=i+1) for i, cf in enumerate(configs)]

    price_df = rrp_df.copy()
    price_df['possible_prices'] = possible_prices
    return price_df

In [114]:
# moved to cls PriceAccessor
def get_possible_prices(cf, df):
    return df[df.sku_config == cf]['possible_prices'].iloc[0]

In [ ]:
group_df = load_group_snapshots(gname, group_dir)

In [113]:
price_df = set_possible_prices(configs, group_df)

SKU with changing black price, use the latest price
              sku_config snapshot_date  black_price
828183  79BF4AAFB6584DGS    2017-07-21         24.9
134674  79BF4AAFB6584DGS    2016-12-30         39.9


__Note:__ Round possible prices to 2 decimals (mtg on 10 Nov 2017)

## Query inventory amounts

In [71]:
def query_inventory(configs, dd, group_stock):
    stock_df = group_stock[group_stock['date'] == dd]
    cols = ['sku_config', 'date', 'stock']
    return stock_df[stock_df.sku_config.isin(configs)][cols].drop_duplicates()

In [62]:
group_stock = my_trigger.query_stock_data(gid)

In [72]:
stock_df = query_inventory(configs, dd, group_stock)

## Merge all data to a single DF

In [115]:
slow_df = pd.merge(price_df, stock_df)
slow_df.sort_values('black_price')

,black_price,sku_config,possible_prices,date,stock
6,24.9,79BF4AAFB6584DGS,"[23.65, 22.41, 21.16]",2017-05-28,72
1,34.9,9EB52AA5DA4D3DGS,"[33.16, 31.41, 29.66]",2017-05-28,32
3,34.9,D4094AA895E68CGS,"[33.16, 31.41, 29.66]",2017-05-28,148
5,34.9,1F22DAA71C149FGS,"[33.16, 31.41, 29.66]",2017-05-28,70
7,34.9,FB590AA19EAE04GS,"[33.16, 31.41, 29.66]",2017-05-28,125
9,34.9,86D69AA83B2A30GS,"[33.16, 31.41, 29.66]",2017-05-28,59
2,39.9,1EF9EAA17AB709GS,"[37.9, 35.91, 33.92]",2017-05-28,157
4,44.9,0AE69AACA1018FGS,"[42.66, 40.41, 38.16]",2017-05-28,113
0,79.9,1376FAAE0DBBF1GS,"[75.9, 71.91, 67.92]",2017-05-28,49
8,79.9,D803EAA23402BFGS,"[75.9, 71.91, 67.92]",2017-05-28,49


## Brute-force with all price settings
Here we compare a brute-force solution (consider all price settings) with the solution provided by CPLEX.

In [261]:
tmp = pd.DataFrame({'price': test_prices})

In [283]:
perm_idx = [[0,0,0], [1,1,1], [2,2,2],
            [0,0,1], [0,0,2], [1,1,0], [1,1,2], [2,2,0], [2,2,1],
            [1,0,0], [2,0,0], [0,1,1], [2,1,1], [0,2,2], [1,2,2],
            [0,1,0], [0,2,0], [1,0,1], [1,2,1], [2,0,2], [2,1,2],
            [0,1,2], [0,2,1], 
            [1,0,2], [1,2,0], 
            [2,0,1], [2,1,0] ]

In [284]:
price_settings = [ list(tmp.price.iloc[perm_idx[i]]) for i in range(len(perm_idx))]

In [294]:
frames = [predict_sales(estimator, dd, ps, count=i+1) for i, ps in enumerate(price_settings)]
res = pd.concat(frames)

price setting 1
price setting 2
price setting 3
price setting 4
price setting 5
price setting 6
price setting 7
price setting 8
price setting 9
price setting 10
price setting 11
price setting 12
price setting 13
price setting 14
price setting 15
price setting 16
price setting 17
price setting 18
price setting 19
price setting 20
price setting 21
price setting 22
price setting 23
price setting 24
price setting 25
price setting 26
price setting 27


In [301]:
res.sort_values(['sum'])

,sku_config,current_price,predict_sales,price_setting,sum,revenue
88572,9EB52AA5DA4D3DGS,29.66,1.402374,3,88.98,41.594414
88715,1F22DAA71C149FGS,29.66,1.402374,3,88.98,41.594414
96431,D4094AA895E68CGS,29.66,1.153103,3,88.98,34.201031
88715,1F22DAA71C149FGS,29.66,1.402374,15,90.73,41.594414
88572,9EB52AA5DA4D3DGS,31.41,1.282444,15,90.73,40.281574
88572,9EB52AA5DA4D3DGS,29.66,1.402374,9,90.73,41.594414
88715,1F22DAA71C149FGS,29.66,1.402374,9,90.73,41.594414
96431,D4094AA895E68CGS,29.66,1.153103,15,90.73,34.201031
96431,D4094AA895E68CGS,31.41,1.144713,9,90.73,35.955429
88572,9EB52AA5DA4D3DGS,29.66,1.402374,21,90.73,41.594414


In [300]:
res['sum'].value_counts()

95.98    18
94.23    18
92.48     9
92.48     9
90.73     9
97.73     9
94.23     3
99.48     3
88.98     3
Name: sum, dtype: int64

In [299]:
res['revenue'] = res['current_price'] * res['predict_sales']
res.groupby(['price_setting', 'sum']).agg({'revenue': sum}).reset_index().sort_values('revenue', ascending=False)

,price_setting,sum,revenue
0,1,99.48,123.010379
10,11,95.98,122.078941
16,17,95.98,122.078941
7,8,92.48,121.147504
3,4,97.73,121.007131
15,16,97.73,120.766101
9,10,97.73,120.766101
25,26,94.23,120.075694
22,23,94.23,120.075694
24,25,94.23,119.834664


# Side Notes

+ There are SKUs with too large black prices. These must be errors from data input.

In [ ]:
print(sum(rrp_df.black_price > 1000))
large_price_skus = rrp_df[rrp_df.black_price > 1000]

In [ ]:
cols = ['sku_config', 'product_name', 'brand_name']
large_price_skus.merge(my_trigger.prod_df[cols]).drop_duplicates()

## Toy test case

In [ ]:
# set up test
configs, prices = ['cf1', 'cf2'], range(5, 20, 5)
n_config, n_price = len(configs), len(prices)
inv_amount = np.random.randint(1, 10, n_config)
today = pd.datetime.today()

min_s, max_s = n_config * min(prices), n_config * max(prices)
s = rand_int_in(min_s, max_s)
today = pd.datetime.today()

In [ ]:
prob = populateLP(s, today, configs, prices, inv_amount, binary=False)
print()
print('Started solver')
prob.solve()
show_sol_info(prob) 

## Check SKUs with weird prices

In [ ]:
# cf = AD9ABAA31B83EFGS
sub_df = group_df.query('sku_config == "AD9ABAA31B83EFGS" and is_visible == 1') .drop_duplicates()
sub_df.sort_values('price', ascending=False, inplace=True)

In [ ]:
sub_df.info()

In [ ]:
cols = ['sku_config', 'price', 'online_aging', 'is_visible', 'snapshot_date']
sub_df[cols].head()

In [ ]:
sub_df.query('price > 10**5').snapshot_date.describe()

In [ ]:
sub_df.price.describe()

## Attach black price to feature matrix

We also need black price data, thus we query them and attach to current feature matrix.

In [160]:
all_configs = group_df.sku_config.dropna().unique()
black_prices = query_black_price(configs=all_configs, group_df=group_df)

SKU with changing black price, use the latest price
          sku_config snapshot_date  black_price
5106  MA193AA24MPFMY    2016-07-11         55.9
2649  MA193AA24MPFMY    2016-05-26         59.9
SKU with changing black price, use the latest price
          sku_config snapshot_date  black_price
5108  MA193AA13MPQMY    2016-07-11        189.0
2651  MA193AA13MPQMY    2016-05-26        179.0
SKU with changing black price, use the latest price
           sku_config snapshot_date  black_price
29772  DO816AA14RUFMY    2016-09-30         93.9
2807   DO816AA14RUFMY    2016-05-31         89.9
SKU with changing black price, use the latest price
            sku_config snapshot_date  black_price
9133  APAKTAA0000080GS    2016-08-04         39.9
3216  APAKTAA0000080GS    2016-06-10         44.9
SKU with changing black price, use the latest price
            sku_config snapshot_date  black_price
536076  MI511AA28IXRMY    2017-06-17        115.0
524769  MI511AA28IXRMY    2017-06-16       1399.0
3785 

In [416]:
gid = ('dresses', 'female', 'autumn-winter')
gname = '_'.join(list(gid))
fname = feat_dir + gname + '.csv'

feat_mat = pd.read_csv(fname)
feat_mat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276768 entries, 0 to 276767
Data columns (total 6 columns):
sku_config           276677 non-null object
snapshot_date        276768 non-null object
current_price        276768 non-null float64
discount_from_rrp    276768 non-null float64
n_competitor         276768 non-null int64
rel_price            276768 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 12.7+ MB


In [417]:
feat_mat.dropna(subset=['sku_config'], inplace=True)
feat_mat = feat_mat.merge(black_prices)

In [418]:
feat_mat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276677 entries, 0 to 276676
Data columns (total 7 columns):
sku_config           276677 non-null object
snapshot_date        276677 non-null object
current_price        276677 non-null float64
discount_from_rrp    276677 non-null float64
n_competitor         276677 non-null int64
rel_price            276677 non-null float64
black_price          276677 non-null float64
dtypes: float64(4), int64(1), object(2)
memory usage: 16.9+ MB


In [419]:
feat_mat.to_csv(fname, index=False)

## Examine solution of LPs

In [ ]:
def show_sol_info(prob):
    numrows = prob.linear_constraints.get_num()
    numcols = prob.variables.get_num()
    print()
    # solution.get_status() returns an integer code
    print("Solution status = ", prob.solution.get_status(), ":", end=' ')
    # which is used to print the corresponding string
    print(prob.solution.status[prob.solution.get_status()])

    print("Solution value  = ", prob.solution.get_objective_value())
    slack = prob.solution.get_linear_slacks()
    pi = prob.solution.get_dual_values()
    x = prob.solution.get_values()
    dj = prob.solution.get_reduced_costs()
    for i in range(numrows):
        print("Row %d:  Slack = %10f  Pi = %10f" % (i, slack[i], pi[i]))
    for j in range(numcols):
        print("Column %d:  Value = %10f Reduced cost = %10f" %
              (j, x[j], dj[j]))

## Old codes

In [42]:
predictions = demand_predictor.predict_by_xgb(best_xgb, test_df)

In [ ]:
# use all models
# predictions = demand_predictor.predict(best_models, test_df)
# print(demand_predictor.sort_models_by_test_rmse())